# Colab — Treinamento de Agente
Breve notebook para configurar Colab, instalar dependências, treinar o agente e salvar checkpoints no Google Drive.

## 1) Instalar dependências
Use as células abaixo para instalar dependências Python necessárias no Colab. Primeiro edite `requirements-colab.txt` se desejar.

In [ ]:
# Atualizar pip e instalar dependências do arquivo
!python -m pip install --upgrade pip
!pip install -r requirements-colab.txt

### (Opcional) Instalar PyTorch com suporte CUDA (se usar GPU no Colab)
Se desejar GPU, comente a célula anterior e execute a célula abaixo ajustando o wheel conforme a versão CUDA do runtime do Colab.

In [ ]:
# Exemplo: instalar PyTorch com CUDA 11.8 (ajuste conforme runtime do Colab)
# !pip install --index-url https://download.pytorch.org/whl/cu118 torch torchvision torchaudio --upgrade
print("Verifique sua versão do CUDA no Colab e escolha o wheel apropriado se necessário.")

## 2) Montar Google Drive e criar pastas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Criar pasta para salvar runs
import os
run_dir = '/content/drive/MyDrive/agent_runs'
os.makedirs(run_dir, exist_ok=True)
print('Runs will be saved to', run_dir)

## 3) Clonar repositório (opcional) e definir diretório de trabalho
Se já subiu os arquivos manualmente, apenas ajuste `WORKDIR`.

In [ ]:
# Clone repo if needed (uncomment to run)
# !git clone https://github.com/devolopbomfim/Xadrez_AI_Final.git
# %cd Xadrez_AI_Final

import os
WORKDIR = '/content/Xadrez_AI_Final'
if os.path.exists(WORKDIR):
    get_ipython().run_line_magic('cd', 
)
else:
    print('Repo not found in', WORKDIR, '- upload or clone it.')

## 4) Checar GPU e ambiente

In [ ]:
# GPU check
!nvidia-smi || echo 'No GPU detected'
import torch
print('torch:', torch.__version__)
print('cuda available:', torch.cuda.is_available())

## 5) Configuração de hiperparâmetros (arquivo YAML)
É gravado em `colab_train_config.yaml` e pode ser editado antes de rodar.

In [ ]:
# Exemplo de config YAML
cfg = '''
learning_rate: 0.001
batch_size: 128
gamma: 0.99
env_name: local_chess
total_timesteps: 100000
seed: 42
checkpoint_interval: 500
device: cuda
iters_per_epoch: 100
'''
with open('colab_train_config.yaml', 'w') as f:
    f.write(cfg)
print('Wrote colab_train_config.yaml')

## 6) Exemplo: Gerar self-play + treinar (run_iteration)
Use a célula abaixo para executar um ciclo completo de self-play -> treino -> avaliação.

In [ ]:
# Ajuste os parâmetros conforme necessário
AGENT_DIR = 'models/AgentA'
!python3 training/run_iteration.py {AGENT_DIR} --iteration 0 --num-selfplay 5 --selfplay-sims 50 --trainer-iters 200 --batch-size 128

## 7) Salvar/Carregar checkpoints no Drive
Exemplo de como copiar checkpoints para o Drive.

In [ ]:
import shutil, glob, os
ckpt_src = 'models/AgentA/checkpoints'
ckpt_dst = '/content/drive/MyDrive/agent_runs/AgentA_checkpoints'
os.makedirs(ckpt_dst, exist_ok=True)
for f in glob.glob(os.path.join(ckpt_src, '*.pt')):
    shutil.copy(f, ckpt_dst)
print('Copied checkpoints to', ckpt_dst)

## 8) TensorBoard (visualizar no Colab)

In [ ]:
# Iniciar tensorboard apontando para pastas de logs
# %load_ext tensorboard
# %tensorboard --logdir models/AgentA/logs --host 0.0.0.0 --port 6006
print('Use TensorBoard cell if logs are available')

## 9) Avaliação e renderização de episódios

In [ ]:
# Rodar um episódio de avaliação rápido (exemplo)
from core.board.board import Board
from training.mcts import MCTS

# Código de avaliação simples (exemplo minimal)
print('Run evaluation using provided scripts (see training/run_iteration.py)')

## 10) Script reutilizável (run_train.sh)

In [ ]:
# Exemplo de script run_train.sh
# mkdir -p runs && python3 training/run_iteration.py models/AgentA --iteration 0 --num-selfplay 10 --trainer-iters 500

## 11) Smoke tests rápidos
Use asserts para garantir que forward pass e um passo de treino rodam.

In [ ]:
# Smoke test: instanciar modelo e fazer forward
import torch
from training.model import make_model
m = make_model(device='cpu')
import numpy as np
x = np.random.randn(1, 13, 8, 8).astype('float32')
t = torch.tensor(x)
pi, v = m(t)
assert pi is not None and v is not None
print('Smoke test passed')